In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [3]:
import pandas as pd
import numpy as np
import sys # 최대 정수
from tensorflow.keras import utils # utils.to_categorical() : 분류분석시 y데이터 원핫인코딩
from tensorflow.keras.models import Sequential # 모델 생성
from tensorflow.keras.layers import Dense, Input
import matplotlib.pyplot as plt

# 로지스틱 회귀분석(이진분류)
## 1. 데이터 셋 생성 & 전처리

In [4]:
# 1. csv파일을 데이터프레임을 읽어오기(결측치처리, 균형확인 용이) 기본encoding=utf-8
df = pd.read_csv('data/pima-indians-diabetes.csv', comment='#', header=None)
df.head()
# 0. 임신횟수
# 1. 2시간내 포도당 농도
# 2. 혈압
# 3. 피부접힘 두께
# 4. 인슐린수치
# 5. BMI체질량지수
# 6. 가족력
# 7. 나이

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [9]:
df.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [10]:
df.isna().sum() # 열별 결측치 갯수

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

In [13]:
# 타겟변수의 균형
df[8].value_counts()/len(df)

0    0.651042
1    0.348958
Name: 8, dtype: float64

In [15]:
df.values
df.to_numpy()

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [6]:
# 2. csv파일을 numpy 배열로 읽어오기 : np.loadtxt() #이 있는 행은 제외하고 읽어오기
        #기본 encoding=cp949
dataset = np.loadtxt('data/pima-indians-diabetes.csv',delimiter=',', encoding='utf-8')
dataset.shape

(768, 9)

In [10]:
# 데이터 분할 : 학습데이터셋(모델학습용) + 시험데이터셋(모델평가용)
X_train = dataset[:620, :-1]
y_train = dataset[:620, -1]
X_test  = dataset[620:, :-1]
y_test  = dataset[620:, -1]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((620, 8), (620,), (148, 8), (148,))

# 2. 모델구성

In [16]:
model = Sequential()
# model.add(Dense(units=32, input_dim=8, #input_shape=(8,)
#                activation='relu'))
model.add(Input(shape=(8,)))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='elu'))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                288       
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833
Trainable params: 833
Non-trainable params: 0
_________________________________________________________________


# 3. 학습과정 설정

In [17]:
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 학습

In [13]:
%%time
hist = model.fit(X_train, y_train, # 훈련데이터셋(학습데이터셋)
                epochs=200,
                batch_size=310,
                # validation_data=(X_test, y_test), 검증데이터셋
                validation_split=0.1, # 검증데이터 비율
                verbose=0) # 로그 출력 X

Epoch 1/200
2/2 [==============================] - 0s 3ms/step - loss: 11.2758 - accuracy: 0.3484
Epoch 2/200
2/2 [==============================] - 0s 3ms/step - loss: 9.1187 - accuracy: 0.3387
Epoch 3/200
2/2 [==============================] - 0s 3ms/step - loss: 7.8938 - accuracy: 0.3645
Epoch 4/200
2/2 [==============================] - 0s 5ms/step - loss: 7.0941 - accuracy: 0.3903
Epoch 5/200
2/2 [==============================] - 0s 2ms/step - loss: 6.3908 - accuracy: 0.4048
Epoch 6/200
2/2 [==============================] - 0s 3ms/step - loss: 5.6069 - accuracy: 0.4355
Epoch 7/200
2/2 [==============================] - 0s 3ms/step - loss: 4.8007 - accuracy: 0.4435
Epoch 8/200
2/2 [==============================] - 0s 4ms/step - loss: 4.0390 - accuracy: 0.4468
Epoch 9/200
2/2 [==============================] - 0s 2ms/step - loss: 3.4437 - accuracy: 0.4516
Epoch 10/200
2/2 [==============================] - 0s 4ms/step - loss: 2.9860 - accuracy: 0.4532
Epoch 11/200
2/2 [==========

2/2 [==============================] - 0s 2ms/step - loss: 0.5803 - accuracy: 0.7065
Epoch 85/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5810 - accuracy: 0.7097
Epoch 86/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5783 - accuracy: 0.7145
Epoch 87/200
2/2 [==============================] - 0s 3ms/step - loss: 0.5757 - accuracy: 0.7097
Epoch 88/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5724 - accuracy: 0.7113
Epoch 89/200
2/2 [==============================] - 0s 6ms/step - loss: 0.5721 - accuracy: 0.7113
Epoch 90/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5691 - accuracy: 0.7145
Epoch 91/200
2/2 [==============================] - 0s 4ms/step - loss: 0.5704 - accuracy: 0.7145
Epoch 92/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5668 - accuracy: 0.7129
Epoch 93/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5651 - accuracy: 0.7226
Epoch 94/200
2/2 [===============

2/2 [==============================] - 0s 2ms/step - loss: 0.5104 - accuracy: 0.7532
Epoch 168/200
2/2 [==============================] - 0s 6ms/step - loss: 0.5082 - accuracy: 0.7516
Epoch 169/200
2/2 [==============================] - 0s 3ms/step - loss: 0.5088 - accuracy: 0.7516
Epoch 170/200
2/2 [==============================] - 0s 3ms/step - loss: 0.5101 - accuracy: 0.7581
Epoch 171/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5087 - accuracy: 0.7565
Epoch 172/200
2/2 [==============================] - 0s 3ms/step - loss: 0.5065 - accuracy: 0.7516
Epoch 173/200
2/2 [==============================] - 0s 3ms/step - loss: 0.5065 - accuracy: 0.7532
Epoch 174/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5065 - accuracy: 0.7565
Epoch 175/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5061 - accuracy: 0.7565
Epoch 176/200
2/2 [==============================] - 0s 2ms/step - loss: 0.5057 - accuracy: 0.7565
Epoch 177/200
2/2 [=====

In [15]:
hist.history.keys()

dict_keys(['loss', 'accuracy'])